In [1]:
import polars as pl
import requests

In [2]:
r = requests.get(
    "http://bustime.mta.info/api/where/vehicles-for-agency/MTABC.json?key=TEST"
).json()
df_oba = pl.from_dicts(r["data"]["list"])
r = requests.get(
    "http://bustime.mta.info/api/where/vehicles-for-agency/MTA NYCT.json?key=TEST"
).json()
# combine the two dataframes
df_oba = pl.concat([df_oba, pl.from_dicts(r["data"]["list"])])

In [3]:
r = requests.get(
    "https://bustime.mta.info/api/siri/vehicle-monitoring.json?key=TEST&version=2"
).json()
parsed = [
    v["MonitoredVehicleJourney"]
    for v in r["Siri"]["ServiceDelivery"]["VehicleMonitoringDelivery"][0][
        "VehicleActivity"
    ]
]
df_siri = pl.from_dicts(
    parsed
    # r["Siri"]["ServiceDelivery"]["VehicleMonitoringDelivery"][0]["VehicleActivity"]
)

In [15]:
df_siri = df_siri.with_columns(
    pl.col("VehicleRef").str.split("_").list.last().alias("vehicle_id"),
    passengers=pl.col("MonitoredCall")
    .struct.field("Extensions")
    .struct.field("Capacities")
    .struct.field("EstimatedPassengerCount"),
)
df_oba = df_oba.with_columns(
    pl.col("tripId").str.splitn("_", 2).struct[1].alias("trip_id"),
    pl.col("vehicleId").str.splitn("_", 2).struct[1].alias("vehicle_id"),
)
# [
#     "VehicleID"
# ]
# create polars series with vehicle IDs

In [ ]:
display(
    df_oba.filter(pl.col("occupancyCount").is_not_null()).height,
    df_siri.filter(pl.col("passengers").is_not_null()).height,
)

1432

1317

In [17]:
# display(df_oba, df_siri)

In [6]:
siri_vehicle_ids = df_siri["vehicle_id"]
oba_vehicle_ids = df_oba["vehicle_id"]


display(
    "OBA difference:", oba_vehicle_ids.filter(~oba_vehicle_ids.is_in(siri_vehicle_ids))
)
# display("difference:", siri_vehicle_ids.filter(~siri_vehicle_ids.is_in(oba_vehicle_ids)))

'OBA difference:'

vehicle_id
str
"""9125"""
"""6092"""
"""9115"""
"""9113"""
"""9109"""
…
"""1207"""
"""1203"""
"""1202"""


In [7]:
filtered_df = df_oba.filter(
    pl.col("lastLocationUpdateTime") != pl.col("lastUpdateTime")
)
# rows are equal
# print(f"Total rows: {len(df)}")
# print(f"Filtered rows: {len(filtered_df)}")
# filtered_df.head(5)
filtered_df

lastLocationUpdateTime,lastUpdateTime,location,occupancyCapacity,occupancyCount,occupancyStatus,phase,status,tripId,vehicleId,vehicle_id
i64,i64,struct[2],i64,i64,i64,str,str,str,str,str


In [8]:
siri_diff = siri_vehicle_ids.filter(~siri_vehicle_ids.is_in(oba_vehicle_ids))
print(f"Siri vehicle IDs not in OBA: {siri_diff}")

Siri vehicle IDs not in OBA: shape: (0,)
Series: 'vehicle_id' [str]
[
]


In [9]:
df_oba["phase"].value_counts()

phase,count
str,u32
"""layover_before""",5
"""in_progress""",1741
"""at_base""",416
"""layover_during""",275
"""deadhead_during""",212
"""spooking""",2
"""deadhead_before""",370


In [10]:
df_oba["status"].value_counts()

status,count
str,u32
"""blockInf+stalled""",18
"""default""",901
"""blockInf""",2090
"""blockInf+deviated""",6
"""stalled""",6


In [9]:
df_oba["occupancyStatus"].value_counts()

occupancyStatus,count
i64,u32
3,6
null,3946
1,1133
5,2
2,129


In [ ]:
df_siri["ProgressRate"].value_counts()

ProgressRate,count
str,u32
"""normalProgress""",3203
"""noProgress""",318
"""unknown""",1


In [ ]:
df_siri["ProgressStatus"].value_counts()

ProgressStatus,count
list[str],u32
"[""layover""]",309
null,3212
"[""spooking""]",1
